In [2]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
import re
import random
from nltk.classify.scikitlearn import SklearnClassifier
import pickle

from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

from nltk.classify import ClassifierI
from statistics import mode

In [3]:
#import the data
df = pd.read_excel('pos.xlsx')
df1  = pd.read_excel('neg.xlsx')

In [4]:
df  = df.reset_index(drop = True)
df = df[0:8000]
df.head()

,target,id,acc,data
0,4,1467822272,ersle,I LOVE @Health4UandPets u guys r the best!!
1,4,1467822273,becca210,im meeting up with one of my besties tonight! ...
2,4,1467822283,Wingman29,"@DaRealSunisaKim Thanks for the Twitter add, S..."
3,4,1467822287,katarinka,Being sick can be really cheap when it hurts t...
4,4,1467822293,_EmilyYoung,@LovesBrooklyn2 he has that effect on everyone


In [5]:
df1 = df1[0:8000] # data size can be varied this was done to same time
df1.head()

,target,id,acc,data
0,0,1467810369,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Karoli,"@nationwideclass no, it's not behaving at all...."


In [11]:
def my_str(x): # allowing to include the usernames at the beginning of he statement
    return re.sub(r'(\s)@\w+', r'\1',str(x['data']))

In [12]:
def my_regex(x):# regex to remove the @ sign so that they usernames don't interfere with the token analysis 
    return ' ' + str(x['data'])

In [13]:
df1['data'] = df1.apply(my_str,axis = 1)# using lambda funtion to apply them to multiple rows
df['data'] = df.apply(my_str,axis = 1)

In [14]:
df1['data'] = df1.apply(my_regex,axis = 1)
df['data'] = df.apply(my_regex,axis = 1)

In [15]:
df1.head()

,target,id,acc,data
0,0,1467810369,_TheSpecialOne_,"http://twitpic.com/2y1zl - Awww, that's a b..."
1,0,1467810672,scotthamilton,is upset that he can't update his Facebook b...
2,0,1467810917,mattycus,I dived many times for the ball. Managed to...
3,0,1467811184,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Karoli,"no, it's not behaving at all. i'm mad. why ..."


In [16]:
df.head()

,target,id,acc,data
0,4,1467822272,ersle,I LOVE u guys r the best!!
1,4,1467822273,becca210,im meeting up with one of my besties tonight...
2,4,1467822283,Wingman29,"Thanks for the Twitter add, Sunisa! I got t..."
3,4,1467822287,katarinka,Being sick can be really cheap when it hurts...
4,4,1467822293,_EmilyYoung,he has that effect on everyone


In [17]:
documents = []
for i in range(0,8000):
    for r in str(df['data'][i]).split('\n'):
        documents.append( (r, "pos") )

for i in range(0,8000):
    for r in str(df1['data'][i]).split('\n'):
        documents.append( (r, "neg") )

In [18]:
documents[-5]

('   the snow hit me in the eye and literally made me cry ', 'neg')

In [19]:
all_words = []

for i in range(0,8000):
    short_pos_words = word_tokenize(str(df['data'][i]))
    for w in short_pos_words:
        all_words.append(w.lower())
        
for i in range(0,8000):    
    short_neg_words = word_tokenize(str(df1['data'][i]))
    for w in short_neg_words:
        all_words.append(w.lower())




In [20]:
for word in all_words:
    if word in stopwords.words('english') or word in string.punctuation:
        all_words.remove(word)

In [21]:
all_words = nltk.FreqDist(all_words)

word_features = list(all_words.keys())[:5000]

def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

featuresets = [(find_features(data), category) for (data, category) in documents]





In [22]:
len(featuresets)

16000

In [23]:
len(all_words)

20638

In [24]:
random.shuffle(featuresets)
training_set = featuresets[2000:]
testing_set =  featuresets[:2000]

classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(20)


Original Naive Bayes Algo accuracy percent: 69.75
Most Informative Features
                 snowing = True              neg : pos    =     24.4 : 1.0
                    sick = True              neg : pos    =     20.7 : 1.0
                 dentist = True              neg : pos    =     15.7 : 1.0
                    sore = True              neg : pos    =     15.4 : 1.0
                headache = True              neg : pos    =     13.8 : 1.0
                 welcome = True              pos : neg    =     13.8 : 1.0
                    snow = True              neg : pos    =     12.8 : 1.0
                     sad = True              neg : pos    =     11.8 : 1.0
                  indeed = True              pos : neg    =     11.6 : 1.0
                horrible = True              neg : pos    =     11.0 : 1.0
                   sucks = True              neg : pos    =     10.8 : 1.0
                  throat = True              neg : pos    =     10.4 : 1.0
                   dying

In [25]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [26]:
###############
save_classifier = open("pickled_algos/originalnaivebayes5k.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

save_classifier = open("pickled_algos/MNB_classifier5k.pickle","wb")
pickle.dump(MNB_classifier, save_classifier)
save_classifier.close()

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

save_classifier = open("pickled_algos/BernoulliNB_classifier5k.pickle","wb")
pickle.dump(BernoulliNB_classifier, save_classifier)
save_classifier.close()

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

save_classifier = open("pickled_algos/LogisticRegression_classifier5k.pickle","wb")
pickle.dump(LogisticRegression_classifier, save_classifier)
save_classifier.close()


LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

save_classifier = open("pickled_algos/LinearSVC_classifier5k.pickle","wb")
pickle.dump(LinearSVC_classifier, save_classifier)
save_classifier.close()

NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)


SGDC_classifier = SklearnClassifier(SGDClassifier())
SGDC_classifier.train(training_set)
print("SGDClassifier accuracy percent:",nltk.classify.accuracy(SGDC_classifier, testing_set)*100)


save_classifier = open("pickled_algos/SGDC_classifier5k.pickle","wb")
pickle.dump(SGDC_classifier, save_classifier)
save_classifier.close()

MNB_classifier accuracy percent: 69.89999999999999
BernoulliNB_classifier accuracy percent: 69.85
LogisticRegression_classifier accuracy percent: 69.0
LinearSVC_classifier accuracy percent: 66.7
NuSVC_classifier accuracy percent: 66.45


C:\Users\Shekhar\Anaconda3\envs\nlp\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier accuracy percent: 66.2


In [28]:
save_featuresets = open("pickled_algos/featuresets.pickle","wb")
pickle.dump(featuresets, save_featuresets)
save_featuresets.close()

In [29]:
save_word_features = open("pickled_algos/word_features5k.pickle","wb")
pickle.dump(word_features, save_word_features)
save_word_features.close()


In [30]:
save_documents = open("pickled_algos/documents.pickle","wb")
pickle.dump(documents, save_documents)
save_documents.close()


In [46]:
import sentiment_mod as s

print(s.sentiment("This movie was awesome! The acting was great, plot was wonderful, and there were pythons...so yea!"))
print(s.sentiment("man u are terrible"))
print(s.sentiment("This movie was bad  The acting was horrible, plot was wonderful, and there were no pythons...so nah!"))
print(s.sentiment("This movie was utter junk. I was dying the whole time"))
print(s.sentiment("hey awesome!!!"))

('pos', 1.0)
('neg', 1.0)
('neg', 1.0)
('neg', 1.0)
('pos', 1.0)
